<a href="https://colab.research.google.com/github/naveenkumar2194/Jarvis-28-SUSTAIN-AI-THON/blob/main/Emotion_Detector_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mouting colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/

# Installing the libraries we need

In [1]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 23.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 

# Data pre-processing

In [3]:
def preprocess_data(texts, labels):

    flatten_texts = []
    flatten_labels = []

    for dialogue, emotion_list in zip(texts, labels):
        for sentence in dialogue:
            flatten_texts.append(sentence)
        flatten_labels.extend(emotion_list)

    return flatten_texts, flatten_labels

In [4]:
from datasets import load_dataset

# Load the daily_dialog dataset
dataset = load_dataset("daily_dialog")

# --------------------------------------------------------------------------------
# Train data
texts_train = dataset['train']['dialog']
labels_train = dataset['train']['emotion']

flatten_texts_train, flatten_labels_train = preprocess_data(texts_train, labels_train)

# --------------------------------------------------------------------------------
# Validation data
texts_validation = dataset['validation']['dialog']
labels_validation = dataset['validation']['emotion']

flatten_texts_validation, flatten_labels_validation = preprocess_data(texts_validation, labels_validation)

# --------------------------------------------------------------------------------
# Test data
texts_test = dataset['test']['dialog']
labels_test = dataset['test']['emotion']

flatten_texts_test, flatten_labels_test = preprocess_data(texts_test, labels_test)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
print(len(labels_train))
print(len(labels_validation))
print(len(labels_test))


print(len(flatten_labels_train))
print(len(flatten_labels_validation))
print(len(flatten_labels_test))

11118
1000
1000
87170
8069
7740


# Creating the model

In [6]:
# from transformers import BertTokenizer
# # Load the BERT tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the DistilBert tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize and encode the input texts for train, validation and test
encoded_texts_train = tokenizer(flatten_texts_train, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
encoded_texts_validation = tokenizer(flatten_texts_validation, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
encoded_texts_test = tokenizer(flatten_texts_test, padding='max_length', truncation=True, max_length=128, return_tensors='pt')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
import torch
from torch.utils.data import DataLoader, TensorDataset

labels_tensor_train = torch.tensor(flatten_labels_train)
labels_tensor_validation = torch.tensor(flatten_labels_validation)
labels_tensor_test = torch.tensor(flatten_labels_test)

dataset_train = TensorDataset(encoded_texts_train.input_ids, encoded_texts_train.attention_mask, labels_tensor_train)
dataset_validation = TensorDataset(encoded_texts_validation.input_ids, encoded_texts_validation.attention_mask, labels_tensor_validation)
dataset_test = TensorDataset(encoded_texts_test.input_ids, encoded_texts_test.attention_mask, labels_tensor_test)

# Create a DataLoader
batch_size = 8
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataloader_validation = DataLoader(dataset_validation, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)


In [8]:
print(flatten_labels_train)

[0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 4, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 6, 0, 0, 0, 0, 0, 0, 4, 4, 0, 4, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 3, 4, 4, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 6, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Train

In [9]:
import torch.nn as nn
from transformers import BertForSequenceClassification
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)
model.to(device)

# Define loss function and optimizer

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Training and validation loop
num_epochs = 10
best_loss = None

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_length = 0
    correct_predictions = 0

    dataloader_train = tqdm(dataloader_train, desc="Training Iteration")

    for batch in dataloader_train:
        input_ids, attention_mask, label = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_loss += loss.item()
        total_length += len(label)

        preds = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (preds == label).sum()

        loss.backward()
        optimizer.step()

    accuracy = correct_predictions.float() / total_length
    avg_loss = total_loss / total_length
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss}, Training Accuracy: {accuracy}")


    model.eval()
    total_val_loss = 0
    total_val_length = 0
    correct_val_predictions = 0

    with torch.no_grad():
        for batch in dataloader_validation:
            input_ids, attention_mask, label = batch

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            label = label.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=label)
            loss = outputs.loss
            total_val_loss += loss.item()
            total_val_length += len(label)

            val_preds = torch.argmax(outputs.logits, dim=1)
            correct_val_predictions += (val_preds == label).sum()

    val_accuracy = correct_val_predictions.float() / total_val_length
    avg_val_loss = total_val_loss / total_val_length
    print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss}, Validation Accuracy: {val_accuracy}")

    scheduler.step(avg_val_loss)

    # Save the model if the validation loss is the best we've seen so far.
    if not best_loss or avg_val_loss < best_loss:
        torch.save(model.state_dict(), 'emotion_detection_model.pt')
        best_loss = avg_val_loss

Using device: cuda


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Training Iteration: 100%|██████████| 10897/10897 [17:31<00:00, 10.36it/s]


Epoch 1/10, Training Loss: 0.05131748067190512, Training Accuracy: 0.8589193820953369
Epoch 1/10, Validation Loss: 0.0322355147672653, Validation Accuracy: 0.9046970009803772


Training Iteration:  85%|████████▍ | 9231/10897 [14:54<02:41, 10.32it/s]


KeyboardInterrupt: 

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_save_path = "distilbert-emotion-detector"

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('distilbert-emotion-detector/tokenizer_config.json',
 'distilbert-emotion-detector/special_tokens_map.json',
 'distilbert-emotion-detector/vocab.txt',
 'distilbert-emotion-detector/added_tokens.json')

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)

# Load the model weights
state_dict = torch.load('emotion_detection_model.pt')

# Apply the weights to the model
model.load_state_dict(state_dict)

model.to(device)

model.eval()
total_test_loss = 0
total_test_length = 0
correct_test_predictions = 0

with torch.no_grad():
    for batch in dataloader_test:
        input_ids, attention_mask, label = batch

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = outputs.loss
        total_test_loss += loss.item()
        total_test_length += len(label)

        test_preds = torch.argmax(outputs.logits, dim=1)
        correct_test_predictions += (test_preds == label).sum()

    test_accuracy = correct_test_predictions.float() / total_test_length
    avg_test_loss = total_test_loss / total_test_length
    print(f"Test Loss: {avg_test_loss}, Test Accuracy: {test_accuracy}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test Loss: 0.05174991291735889, Test Accuracy: 0.8543927073478699
